# Compute r5 traveltimes
Angelehnt an https://github.com/DigitalGeographyLab/travel-time-matrix-computer/tree/main/src/travel_time_matrix_computer

In [1]:
import configparser
import os
import sys
import geopandas as gpd
from datetime import datetime
import pandas as pd
from time import time, localtime
import logging
from pathlib import Path
import pandas as pd
import glob

# del sys.modules["r5_compute_travel_time_matrixes"]
from r5_compute_travel_time_matrixes import (
    create_transport_network,
    calculate_travel_time_matrix,
    load_params_from_ini,
    calculate_detailed_iternaries_matrix,
    buffer_geometries,
    get_logger
)

In [2]:
# pfade aus config holen
# init configParser and open file
repoDirectoryPath = ''

# gemeinden kreise laden
kreise = gpd.GeoDataFrame.from_file(os.path.join(repoDirectoryPath, ''))
segeberg = kreise[kreise['GEN'] == 'Segeberg']
hamburg = kreise[kreise['GEN'] == 'Hamburg']

# gitter (ORIGINS) laden
Gitter_MRH = gpd.GeoDataFrame.from_file(os.path.join(repoDirectoryPath, '')).set_index('GITTER_ID_100m') 

# ärzte (DESTINATIONS) laden
# Here load your destinations in gdf
source_gdf = gpd.GeoDataFrame.from_file(os.path.join(repoDirectoryPath, ''))

In [ ]:
##############################################
# Routing  #
##############################################

# LOGGER #
logger = get_logger('./logs')

# Allow 6 GB memory #
sys.argv.append(["--max-memory", "6G"])

# Kreise #
kreise_mrh = kreise.copy()

# Parameter aus Config laden
general_params, params = load_params_from_ini('./config.ini')

logger.info('Starting skript ..')
start_time = time()

for index, row in kreise_mrh.iterrows():
    kreis = gpd.GeoDataFrame([row], columns=kreise.columns, crs=kreise.crs)
    name_kreis = kreis['GEN'].iloc[0]


    ######################
    # Param - OutputPath #
    ######################
    transport_modes = '_'.join([param['mode'] for param in params])
    out_folder = os.path.join(repoDirectoryPath, '', transport_modes)
    if not Path(out_folder).is_dir():
        os.mkdir(out_folder)
    name_output_file = f'travelTimes_{name_kreis}_{transport_modes}_Zensus2022'


    ########
    # Rest #
    ########
    kreis_buffered = buffer_geometries(gdf=kreis, 
                                        buffer_distance_km=general_params['buffer_distance_km'], 
                                        hamburg_gdf=kreise_mrh[kreise_mrh['GEN'] == 'Hamburg'], 
                                        buffer_hamburg=general_params['buffer_hamburg_km'])

    logger.info('')
    logger.info(f'Kreis: {name_kreis}')
    logger.info(f'Uhrzeit: {localtime(time()).tm_hour}:{localtime(time()).tm_min}')

    # origins/destinations definieren
    origins = Gitter_MRH[Gitter_MRH.geometry.within(kreis.geometry.iloc[0])].copy()
    destinations = source_gdf[source_gdf.geometry.within(kreis_buffered.geometry.iloc[0])].copy()

    try:
        transport_network = create_transport_network(repoDirectoryPath=repoDirectoryPath, 
                                                        logger=logger, 
                                                        kreis=kreis)
    except Exception as e:
        logger.warning(f'Feher beim erstellen des transport_network für {name_kreis};  {e}')

    start_time2 = time()
    try:
        for departure in general_params['departure_times']:
            out_folder = os.path.join(out_folder, f'{departure.strftime("%d%m%Y_%H%M")}')
            if not Path(out_folder).is_dir():
                os.mkdir(out_folder)

            travel_times = calculate_travel_time_matrix(
                transport_network = transport_network,
                origins_grid = origins,
                destinations = destinations,
                departure = departure,
                out_path = os.path.join(out_folder, f'{name_output_file}_{departure.strftime("%d%m%Y_%H%M")}'),
                params = params,
                speeds = True,
                logger=logger
            )
    except Exception as e:
        logger.warning(f'Feher beim erstellen der TravelTimeMatrix für {name_kreis} für {departure};  {e}')

    logger.info(f'TravelTimeMatrix für {name_kreis} erstellt;  Zeit: {round((time()-start_time2)/60, 2)} min')

logger.info(f'Berechnungen abgeschlossen;  Zeit: {round((time()-start_time)/60, 2)} min')

## Explore Results

In [ ]:
name_kreis = 'Stade'

kreis_i = kreise[kreise['GEN'] == name_kreis]
kreis_i_buffered = buffer_geometries(gdf=kreis_i, buffer_distance_km=20, hamburg_gdf=hamburg, buffer_hamburg=5)
source_gdf_i = source_gdf[source_gdf.geometry.within(kreis_i_buffered.geometry.iloc[0])]
bus_stops = gpd.GeoDataFrame.from_file(os.path.join(repoDirectoryPath, f'.gpkg'))
bus_stops = bus_stops[bus_stops.geometry.within(kreis_i_buffered.geometry.iloc[0])]

travel_times = pd.read_csv(os.path.join(repoDirectoryPath, f'/travelTimes_{name_kreis}_TRANSIT_Zensus2022_22102024_1200_1:00:00.csv'))
if not 'id' in Gitter_MRH.columns:
    Gitter_MRH['id'] = Gitter_MRH.index

min_times = travel_times.groupby("from_id")['TRANSIT_avg_p1'].min()
gitter = Gitter_MRH[Gitter_MRH.geometry.within(kreis_i.geometry.iloc[0])]
merge_min_times = gitter.merge(min_times.reset_index(), left_on="id", right_on="from_id",how='left')

map = kreise.explore(color='black', style_kwds= {'fill':False}, tiles=None)
map = kreis_i_buffered.explore(m=map, color='red', style_kwds= {'fill':False})
bins = [10, 20, 30, 60, 90, 120]
map = merge_min_times.explore(
    column="TRANSIT_avg_p1",
    style_kwds={"stroke":False},
    cmap="viridis_r",
    scheme="user_defined",
    classification_kwds={"bins": bins},
    m=map,
    legend=True,
    legend_kwds={
        "labels": ["0-10", "11-20", "21-30", "31-60", "61-90", "91-120"],
        "colorbar": False
    },
    missing_kwds={"color": "red", "label": "> 120"},
    popup=True
)
map = source_gdf_i.explore(m=map, color = 'green')
map = bus_stops.explore(m=map, color = 'orange', marker_kwds={'radius':1})
map